# Chapter 6 - Testing and Debugging

Our programs don't always function properly the first time we run it. Testing is the process of running a program to try and ascertain whether or not it works as intended. Debugging is the process of trying to fix a program that you already know does not work as intended. 

Good programmers design their programs in ways that make them easier to test and debug. The key to doing this is breaking the program up into separate components that can be implemented, tested, and debugged independently of other components. 

Up until now, we have only discussed one mechanism for modularizing programs, the function. When we get to other mechanism, in particular classes, we will return to some of the topics covered in this chapter.

## 6.1 Testing

The most important thing to say about testing is that its purpose is to show that bug exist, not to show that a program is bug free. To quote Edsger Dijkstra, "Program testing can be used to show the presence of bugs, but never to show their absence !".

Why is this so? Even the simplest programs has billions of possible inputs. Consider, for example:

In [ ]:
def isBigger(x,y):
    """Assumes x and y are int. Returns True if x is less than y and False otherwise."""

Running it on all possible pairs of integers would be tedious. The best we can do is to run it on pairs of integers that have a reasonable probability of producing the wrong answer if there is a bug on the program. 

The key to testing is finding a collection of inputs, called a test suite, that has a high likelihood of revealing bugs, yet does not take too long to run. The key to doing this is partitioning the space of all possible inputs into subsets that provide equivalent information about the correctness of the program and then constructing a test suite that contains at least one input from each partition. Usually, constructing such a test suite is not actually possible. Think of this as an unachievable ideal. 

A partition of a set divides the set into a collection of subsets such that each element of the original set belong to exactly one of the subsets, i.e. exhaustive and mutually exclusive. Consider, for example, isBigger(x,y). The set of possible inputs is all pairwise combinations of integers. One way to partition this set is into these seven subsets:

x positive, y positive x negative, y negative
x positive, y negative x negative, y positive
x = 0, y = 0 x = 0, y ≠ 0 x ≠ 0, y = 0

If one tested the implementation on at least one value from each of these subsets, there would be reasonable probability of exposing a big if one exist.

For most programs, finding a good partitioning of the inputs is far easier said than done. Typically, people rely on heuristics based on exploring different paths through some combination of the code and the specifications. Heuristics based on exploring paths through the code fall into a class called glass-box testing. Heuristics based on exploring paths through the specifications fall into a class called black-box testing.

### 6.1.1 Black Box Testing

In principle black-box tests are constructed without looking at the code to be tested. Black-box testing allows testers and implementers to be drawn from separate populations. This independences reduce the likelihood of generating test suites that exhibit mistakes that are correlated with mistakes in the code. Suppose, a programmer make the implicit, but invalid, assumption that a function would never be called with a negative number. If the same person constructed the test suite for the program, he would likely repeat the mistake, and not test the function with a negative argument.

A good way to generate black-box test data is to explore paths through a specification. Consider the specification:

In [ ]:
def sqrt(x,epsilon):
    """Assumes x and epsilon are floats. x>=0, epsilon>0. Returns result such that 
       x-epsilon <= result**2 <= x+epsilon."""

For sqrt, it might make sense to try some of the values as shown below:

![](sqrt_blackbox.jpg)

The first four rows are intended to represent typical cases which include zero, perfect square, number less than one and number with irrational square root. The remaining rows test extremely large and small values of x and epsilon. If any of these tests fail, then something need to be fixed. Perhaps there is a bug or perhaps the spesification needs to be changed so that it is easier to meet. For example, it might be unreasonable to expect to find an approximation of a square root with epsilon that is ridiculously small.

Another important boundary condition to think about is aliasing. Consider, for example, the code:

In [ ]:
def copy(L1,L2):
    """Assumes L1,L2 are lists. Mutates L2 to be a copy of L1.""" 
    while len(L2)>0:
        #While L2 isn't an empty list, keep delete L2's last element
        L2.pop() #remove last element of L2
        
    #At this point L2 is already an empty list.
    for e in L1:
        #append L1's element to initially empty L2
        L2.append(e)

In [ ]:
L1 = [1,2]
L2 = [4,5,6]

print('L1 =', L1)
print('L2 =', L2)

copy(L1,L2)

print('L2 =', L2)

### Glass-box Testing

Black box testing is rarely sufficient. Without looking at the internal structure of the code, it is impossible to know which test cases are likely to provide information. Consider the trivial example:

In [ ]:
def isPrime(x):
    """Assumes that x>=0 is an integer. Returns True if x is prime; False otherwise."""
    if x<=2:
        #special cases
        return False
    
    #if x>2
    for i in range(2,x):
        print('i =',i)
        if x%i == 0:
            return False
    return True

Without looking at the code, one might not test isPrime(2) and would therefore not discover that the function isPrime(2) return False. Glass-box test suites are usually much easier to construct than black-box test suites. The notion of a path through code is well defined.

A glass-box test suites is path-complete if it exercises every potential path through the program. This is typically impossible to achieve. Furthermore, even a path-complete test suite does not guarantee that all bugs will be exposed. Consider:

In [ ]:
def abs(x):
    """Assumes x is an integer. Returns x if x>=0 and -x otherwise."""
    if x < -1:
        return -x
    else:
        #x>=0
        return x

Since there are two possible cases, x either positive or it isn't, this might suggest that the set of inputs {-2,2} is sufficicent to explore all paths in the specification. However this test suite will not expose the fact that abs(-1) will return -1.

There are a few rules of thumb that are usually worth following:
* Exercise both branches of all if statements.
* Make sure that each except clause is executed.
* For each for loop, have test cases in which:
        - The loop is not entered.
        - The body of the loop is executed exactly once.
        - The body of the loop is executed more than once.
* For each while loop:
        - Look at the same kind of cases when dealing with for loops.
        - Include test cases corresponding to all possible ways of exiting the loop.
* For recursive functions, include test cases that cause the function to return with no recursive calls, exactly one recursive call and more than one recursive calls.
        

### 6.1.3 Conducting Test

Testing is occuring in two phases. In unit testing, testers constructs and run tests designed to ascertain whether individual units of code work properly. This is followed by integration testing, which is designed to ascertain whether the program as a whole behaves as intended. It is common that failures during integration lead to making changes to invidual unit.

# 6.2 Debugging

There is an urban legend anout how the process of fixing flaws in software came to be known as debugging. The following photo is of September 9, 1947, page in a laboratory book from the group working on the Mark 2 Aiken Relay Calculator at Harvard University.

![](original_bug.jpg)

Some have claimed that the discovery of that unfortunate moth trapped in the Mark 2 led to the use of the phrase debugging. Note that if your program has a bug, It is because you put it there. Runtime bugs can be categorized along two dimensions:

* Overt -> Covert : An overt bugs has an obvious manifestation (program crashes or takes far longer to run). A covert bug has no obvious manifestation, other than providing an incorrect answer. Many bugs fall between the two extremes.
* Persistent -> Intermittent : A persistent bugs occurs every time the program is run with the same inputs. An intermittent bug only occurs some time, even when the program is run on the same inputs and seemingly under the same condition. 

The best kind og bugs to have is overt and persistent. Good programmers try to write their programs in such a way that programming mistakes lead to bugs that are both overt and persistent. This is often called defensive programming. 

Bugs that are both covert and intermittent are almost always the hardest to find and fix.

### 6.2.1 Learning to Debug

Debugging is a learned skill. The good news is that it's not hard to learn and it is a transferable skill. The same skills used to debug software can be used to find out what is wrong with other complex systems, e.g. laboratory experiments or sick humans.

Debugging start when testing has demonstrated that the program behaves in undesirable ways. Debugging is the process of searching for an explanation of that behaviour. The key to being consistently good at debugging is being systematic in conducting that search. 

Start by studying the available data. This include the test results and the program test. Study of all the test results.

Next, form a hypothesis that you believe to be consistent with all the data. The hypothesis could be as narrow as "if i change line 403 from x<y to x<=y, the problem will go away."

Next, design and run a repeatable experiment with the potential to refute the hypothesis. For example, you have a hypothesis that "my program is not terminating because i have the wrong exit condition in some while loop." You might put a print statement before and after each while loop. If these are always paired, then the hypothesis that a while loop is causing nontermination has been refuted. 

Finally, it's important to keep a record of what experiments you have tried.

### 6.2.2 Designing the Experiment

Think of debugging as a search process and each experiment as an attempt to reduce the size of the search space. One way to reduce the size of the search space is to design an experiment that can be used to decide whether a spesific region of code is responsible for a problem uncovered during integration testing. Another way is to reduce the amount of test data needed to provoke a manifestation of a bug. Imagine that you wrote the palindrome checking code as follow:

In [1]:
def isPal(x):
    """Assumes x is a list. Return True if the list is a palindrome; False otherwis."""
    temp = x
    temp.reverse #reverse the order of the element in temp
    if temp == x:
        return True
    else:
        return False

def silly(n):
    """Assumes n>0 is an integer. Gets n input from user. Print 'Yes' if the sequence of inputs forms a palindrome;
       'No' otherwise."""
    for i in range(n):
        result = []
        elem = input('Enter element: ')
        result.append(elem)
    
    if isPal(result):
        print('Yes')
    else:
        print('No')    

In [2]:
silly(2)

Enter element: a
Enter element: b
Yes


Turns out the code is fail for a very simple test. Let's start to systematically reduce the search space. Often the best way to do this is to conduct a bisection search. Find some point about halfway through the code, and devise an experiment that will allow you to decide if there is a problem before that point that might be related to the symptom. 

Looking at silly(), the halfway point is around the line if isPal(result). The obvious thing to check is whether result has the expected value ['a','b']. We check this by inserting the statement print(result) before the if statement in silly. 

In [3]:
def silly(n):
    """Assumes n>0 is an integer. Gets n input from user. Print 'Yes' if the sequence of inputs forms a palindrome;
       'No' otherwise."""
    
    result = []
    
    for i in range(n):
        elem = input('Enter element: ')
        result.append(elem)
    
    print(result)
    if isPal(result):
        print('Yes')
    else:
        print('No')  

In [4]:
silly(2)

Enter element: a
Enter element: b
['a', 'b']
Yes


Result has the correct value but still give the anser 'Yes'. Now we have the reason to belive that problem lies after the print statement. Let's insert the line print(temp,x) before if temp == x.

In [6]:
def isPal(x):
    """Assumes x is a list. Return True if the list is a palindrome; False otherwis."""
    temp = x
    temp.reverse #reverse the order of the element in temp
    
    print('x=',x)
    print('temp_reverse=',temp)
    
    if temp == x:
        return True
    else:
        return False

In [7]:
silly(2)

Enter element: a
Enter element: b
['a', 'b']
x= ['a', 'b']
temp_reverse= ['a', 'b']
Yes


The term does not reverse after temp.reverse .

In [12]:
def isPal(x):
    """Assumes x is a list. Return True if the list is a palindrome; False otherwis."""
    temp = x       #aliasing happen here.
    temp.reverse() #reverse the order of the element in temp
    
    print('x=',x)
    print('temp_reverse=',temp)
    
    if temp == x:
        return True
    else:
        return False

In [13]:
silly(2)

Enter element: a
Enter element: b
['a', 'b']
x= ['b', 'a']
temp_reverse= ['b', 'a']
Yes


Now both temp and x is reversed. It seems that temp.reverse() unexpectedly changed the value of x. An aliasing bug has bitten us: temp and x are names for the same list, both before and after the list get reversed. Let's fix this:

In [14]:
def isPal(x):
    """Assumes x is a list. Return True if the list is a palindrome; False otherwis."""
    temp = x[:]    #temp as a copy of x, thus avoiding aliasing.
    temp.reverse() #reverse the order of the element in temp
    
    print('temp_reverse=',temp)
    print('x=',x)
    
    if temp == x:
        return True
    else:
        return False

In [15]:
silly(2)

Enter element: a
Enter element: b
['a', 'b']
temp_reverse= ['b', 'a']
x= ['a', 'b']
No


### 6.2.3 When the Going Gets Tough

A few pragmatic hints about what to do when debugging gets tough:

* Look for the usual suspects.
* Ask yourself why it is doing what it is.
* The bug is probably not where you think it is.
* Walk away and try again tomorrow.

### 6.2.4 When You have Found the Bug